In [1]:
import tensorflow as tf
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


using the famous FER-2013 as dataset to train and test model

In [2]:

Datadirectory='/content/drive/MyDrive/train/'
# path=os.path.join(Datadirectory,"")
# print(path)
Classes=['angry','disgust','fear','happy','neutral','sad','surprise']


Changing the size of image from 48X48 to 224X224 to easily put it in the model(now doing it only for whole dataset)

In [3]:
training_data=[]
def create_training_data():
  for Category in Classes:
    path=os.path.join(Datadirectory,Category)
    class_num=Classes.index(Category)
    for img in os.listdir(path):
      img_arr=cv2.imread(os.path.join(path,img))
      new_array=cv2.resize(img_arr,(224,224))
      training_data.append([new_array,class_num])

create_training_data()


In [4]:
import random
random.shuffle(training_data)
X=[]
Y=[]
for features,label in training_data:
  X.append(features)
  Y.append(label)
X=np.array(X).reshape(-1,224,224,3) #changing dimensions to properly allow model to work



In [5]:
X=X/255 #normalizing data

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
model=tf.keras.applications.MobileNetV2() #use MobileNetV2 archutecture pre-training model, as it being light weight we can use it for live demo



14548992/14536120 [==============================] - 0s 0us/step


Doing transfer learning to make the model learn classes from last step, and hence we won't have to train the whole model from start, and we can use the already existing weights and biases

In [9]:
base_input=model.layers[0].input

base_output=model.layers[-2].output #removes the last layer


#  Adding more layers at end of model to perform classification of classes
final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(7,activation='softmax')(final_output)

In [11]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
new_model=keras.Model(inputs=base_input,outputs=final_output)
# Freezing the pre-trained-layers
for layer in new_model.layers[:-4]:
    layer.trainable = False
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
new_model.fit(X,Y,epochs=80)
new_model.save('model.h5')

In [20]:
new_model.save("model.h5")